In [1]:
import torch
import os
# os.environ["HF_HOME"] = "./cache"
# os.environ["TRANSFORMERS_CACHE"] = "./cache_transformers"
from time import time
from diff3f import get_features_per_vertex
from utils import convert_mesh_container_to_torch_mesh, cosine_similarity, double_plot, get_colors, generate_colors
from dataloaders.mesh_container import MeshContainer
from diffusion import init_pipe
from dino import init_dino
from functional_map import compute_surface_map

In [7]:
device = torch.device('cuda:0')
torch.cuda.set_device(device)
num_views = 4  # must be a square number
H = 512
W = 512
num_images_per_prompt = 1
tolerance = 0.004
random_seed = 42
use_normal_map = True

In [3]:
def compute_features(device, pipe, dino_model, m, prompt):
    mesh = convert_mesh_container_to_torch_mesh(m, device=device, is_tosca=False)
    mesh_vertices = mesh.verts_list()[0]
    features = get_features_per_vertex(
        device=device,
        pipe=pipe, 
        dino_model=dino_model,
        mesh=mesh,
        prompt=prompt,
        mesh_vertices=mesh_vertices,
        num_views=num_views,
        H=H,
        W=W,
        tolerance=tolerance,
        num_images_per_prompt=num_images_per_prompt,
        use_normal_map=use_normal_map,
    )
    return features.cpu()

In [4]:
pipe = init_pipe(device)
dino_model = init_dino(device)

/opt/conda/envs/diff3f/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/envs/diff3f/lib/python3.10/site-packages/diffusers/configuration_utils.py:239: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a model, please use <class 'unet_2d_condition.UNet2DConditionModel'>.load_config(...) followed by <class 'unet_2d_condition.UNet2DConditionModel'>.from_config(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'pipeline_controlnet_img2img.StableDiffusionControlNetImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:27: UserWarning: xFormers is availab

In [5]:
source_file_path = "../fish/3d_fish_reduced/reference_fish_smpl.glb"
target_file_path =  "../fish/3d_fish_reduced/fish_0118_smpl.glb"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)

In [8]:
f_source = compute_features(device, pipe, dino_model, source_mesh, "fish")

Rendering complete


/opt/conda/envs/diff3f/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 4/4 [00:21<00:00,  5.48s/it]


Number of missing features:  6709
Copied features from nearest vertices
Time taken in mins:  0.43399264812469485


In [9]:
f_target = compute_features(device, pipe, dino_model, target_mesh, "fish")

Rendering complete


100%|██████████| 4/4 [00:21<00:00,  5.32s/it]


Number of missing features:  14918
Copied features from nearest vertices
Time taken in mins:  0.4407424330711365


In [10]:
s = cosine_similarity(f_source.to(device),f_target.to(device))
s = torch.argmax(s, dim=0).cpu().numpy()
cmap_source = get_colors(source_mesh.vert); cmap_target = cmap_source[s]

100%|██████████| 2/2 [00:02<00:00,  1.37s/it]


In [12]:
import numpy as np
double_plot(source_mesh,target_mesh,np.asarray(cmap_source),np.asarray(cmap_target))  

/opt/conda/envs/diff3f/lib/python3.10/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "uint32" does not match required type "float64". A coerced copy has been created.
  warnings.warn(
/opt/conda/envs/diff3f/lib/python3.10/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float32" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


# Apply functional map on features

In [24]:
source_file_path = "meshes/cat.off"
target_file_path = "meshes/lion.off"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)
f_source = compute_features(device, pipe, dino_model, source_mesh, "cat")
f_target = compute_features(device, pipe, dino_model, target_mesh, "lion")

Rendering complete


100%|██████████| 100/100 [03:17<00:00,  1.97s/it]


Number of missing features:  140
Time taken in mins:  3.461587341626485
Rendering complete


100%|██████████| 100/100 [03:14<00:00,  1.94s/it]

Number of missing features:  17
Time taken in mins:  3.356095306078593


In [28]:
s = cosine_similarity(f_source.to(device),f_target.to(device))
s = torch.argmax(s, dim=0).cpu().numpy()
cmap_source = get_colors(source_mesh.vert); cmap_target = cmap_source[s]
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

In [25]:
surface_map = compute_surface_map(source_file_path, target_file_path, f_source.numpy(), f_target.numpy())

mesh1 (7207, 3)
mesh2 (5000, 3)

Computing Laplacian spectrum
Computing 200 eigenvectors
	Done in 0.86 s
Computing 200 eigenvectors
	Done in 0.58 s

Computing descriptors
	Normalizing descriptors

	2048 out of 2048 possible descriptors kept
Computing commutativity operators
	Scaling LBO commutativity weight by 8.5e-10

Optimization :
	50 Ev on source - 50 Ev on Target
	Using 2048 Descriptors
	Hyperparameters :
		Descriptors preservation :1.0e+00
		Descriptors commutativity :1.0e-01
		Laplacian commutativity :1.0e-02
		Orientation preservation :0.0e+00

	Task : CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH, funcall : 16, nit : 13, warnflag : 0
	Done in 4.55 seconds


In [26]:
cmap_source = get_colors(source_mesh.vert); cmap_target = cmap_source[surface_map.cpu().numpy()]
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

# Part segmentation

In [29]:
from sklearn.cluster import KMeans
import numpy as np

k = 6

kmeans = KMeans(n_clusters=k, random_state=0, n_init="auto").fit(f_source)

segments1 = kmeans.predict(f_source)

# Apply centroids on another mesh to segment it in a corresponding manner
segments2 = kmeans.predict(f_target)

In [30]:
segment_colors = generate_colors(k)
cmap_source = np.array([segment_colors[j] for j in segments1])
cmap_target = np.array([segment_colors[j] for j in segments2])

In [31]:
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

In [39]:
source_file_path = "meshes/posed_human.off"
target_file_path = "meshes/cat.off"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)
f_source = compute_features(device, pipe, dino_model, source_mesh, "naked human")
f_target = compute_features(device, pipe, dino_model, target_mesh, "cat")

Rendering complete


100%|██████████| 100/100 [03:13<00:00,  1.93s/it]


Number of missing features:  3
Time taken in mins:  3.3452319582303365
Rendering complete


100%|██████████| 100/100 [03:17<00:00,  1.97s/it]

Number of missing features:  140
Time taken in mins:  3.4606642444928486


In [40]:
k = 6

kmeans = KMeans(n_clusters=k, random_state=0, n_init="auto").fit(f_source)

segments1 = kmeans.predict(f_source)

# Apply centroids on another mesh to segment it in a corresponding manner

segments2 = kmeans.predict(f_target)
segment_colors = generate_colors(k)
cmap_source = np.array([segment_colors[j] for j in segments1])
cmap_target = np.array([segment_colors[j] for j in segments2])

In [41]:
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  